In [1]:
import numpy as np
import random

# ----------------------------
# Helper Functions
# ----------------------------

def distance_matrix(coords):
    """Compute Euclidean distance matrix between all locations."""
    n = len(coords)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist[i][j] = np.linalg.norm(np.array(coords[i]) - np.array(coords[j]))
    return dist

def route_distance(route, dist_matrix):
    """Compute total distance for a given route (including return to depot)."""
    total_dist = 0
    for i in range(len(route) - 1):
        total_dist += dist_matrix[route[i]][route[i+1]]
    total_dist += dist_matrix[route[-1]][route[0]]  # return to depot
    return total_dist

def fitness_function(route, dist_matrix):
    """Fitness = total route distance (we minimize this)."""
    return route_distance(route, dist_matrix)

# ----------------------------
# Grey Wolf Optimizer for TSP
# ----------------------------

def GWO_TSP(num_wolves, max_iter, coords):
    """Grey Wolf Optimizer for Waste Collection Route Optimization."""

    num_points = len(coords)
    dist_matrix = distance_matrix(coords)

    # --- Initialize wolves (random routes)
    wolves = [random.sample(range(num_points), num_points) for _ in range(num_wolves)]
    fitness = [fitness_function(w, dist_matrix) for w in wolves]

    # --- Identify alpha, beta, delta (best three wolves)
    sorted_idx = np.argsort(fitness)
    alpha, beta, delta = [wolves[i] for i in sorted_idx[:3]]
    alpha_score, beta_score, delta_score = [fitness[i] for i in sorted_idx[:3]]

    # --- Main optimization loop
    for t in range(max_iter):
        a = 2 - (2 * t / max_iter)  # Decreasing parameter

        for i in range(num_wolves):
            # Convert routes to numerical positions for manipulation
            X = np.array(wolves[i], dtype=float)

            # Calculate influence from alpha, beta, delta
            for leader, leader_score in zip([alpha, beta, delta], [alpha_score, beta_score, delta_score]):
                r1, r2 = random.random(), random.random()
                A = 2 * a * r1 - a
                C = 2 * r2
                D = abs(C * np.array(leader, dtype=float) - X)
                X = np.array(leader, dtype=float) - A * D

            # Convert back to a valid route using rank-based sorting
            order = np.argsort(X)
            new_route = list(order)
            new_fitness = fitness_function(new_route, dist_matrix)

            # Update wolf
            if new_fitness < fitness[i]:
                wolves[i] = new_route
                fitness[i] = new_fitness

        # --- Update alpha, beta, delta
        sorted_idx = np.argsort(fitness)
        alpha, beta, delta = [wolves[i] for i in sorted_idx[:3]]
        alpha_score, beta_score, delta_score = [fitness[i] for i in sorted_idx[:3]]

        # Optional: print progress
        if (t + 1) % 10 == 0 or t == 0:
            print(f"Iteration {t+1}/{max_iter}, Best Distance: {alpha_score:.2f}")

    return alpha, alpha_score

# ----------------------------
# Example Usage
# ----------------------------

# Coordinates of bins (x, y) — including depot at index 0
coords = [
    (0, 0),   # depot
    (2, 3),
    (5, 2),
    (6, 6),
    (8, 3),
    (1, 7),
    (3, 8)
]

best_route, best_distance = GWO_TSP(num_wolves=20, max_iter=100, coords=coords)

print("\nOptimal Waste Collection Route:")
print(" -> ".join(map(str, best_route + [best_route[0]])))
print(f"Total Distance: {best_distance:.2f}")


Iteration 1/100, Best Distance: 29.41
Iteration 10/100, Best Distance: 28.19
Iteration 20/100, Best Distance: 25.72
Iteration 30/100, Best Distance: 25.72
Iteration 40/100, Best Distance: 25.72
Iteration 50/100, Best Distance: 25.72
Iteration 60/100, Best Distance: 25.72
Iteration 70/100, Best Distance: 25.72
Iteration 80/100, Best Distance: 25.72
Iteration 90/100, Best Distance: 25.72
Iteration 100/100, Best Distance: 25.72

Optimal Waste Collection Route:
6 -> 5 -> 1 -> 0 -> 2 -> 4 -> 3 -> 6
Total Distance: 25.72
